<a href="https://colab.research.google.com/github/andryll/Cortisol-Estimator-with-DL/blob/main/notebooks/CortIncepTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aeon > /dev/null

## Carregando o repositório do GitHub com os dados

In [2]:
!git clone https://github.com/andryll/Cortisol-Estimator-with-DL

Cloning into 'Cortisol-Estimator-with-DL'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 41 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (41/41), 2.15 MiB | 4.24 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU(s) detected: {gpus}")
    tf.config.experimental.set_memory_growth(gpus[0], True)
else:
    print("No GPU detected. Please ensure your Colab runtime is set to 'GPU' (Runtime -> Change runtime type -> T4 GPU).")

No GPU detected. Please ensure your Colab runtime is set to 'GPU' (Runtime -> Change runtime type -> T4 GPU).


## Carregando e Preparando os Dados


1.   Os dados de treino, teste e validação são carregados e concatenados, de modo a desfazer a divisão feita anteriormente.

2.   Em seguida, os dados de teste são normalizados com o MinMaxScaler



In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

data_X = np.load("/home/andrilo/RNAP/Cortisol-Estimator-with-DL/data/X_train.npy")
data_X = np.concatenate((data_X, np.load("/home/andrilo/RNAP/Cortisol-Estimator-with-DL/data/X_val.npy")), axis=0)
data_X = np.concatenate((data_X, np.load("/home/andrilo/RNAP/Cortisol-Estimator-with-DL/data/X_test.npy")), axis=0)
print("data_X shape:", data_X.shape)

data_y = np.load("/home/andrilo/RNAP/Cortisol-Estimator-with-DL/data/y_train.npy")
data_y = np.concatenate((data_y, np.load("/home/andrilo/RNAP/Cortisol-Estimator-with-DL/data/y_val.npy")), axis=0)
data_y = np.concatenate((data_y, np.load("/home/andrilo/RNAP/Cortisol-Estimator-with-DL/data/y_test.npy")), axis=0)

# scaler = MinMaxScaler()
# data_y = scaler.fit_transform(data_y.reshape(-1, 1)).flatten()

print("data_y shape:", data_y.shape)

data_X shape: (148, 587, 4)
data_y shape: (148,)


## Realizando a Regressão com o InceptionTime

In [4]:
from aeon.regression.deep_learning import InceptionTimeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping # Importar EarlyStopping

# Realizando o KFold split com 10 folds
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mses = [] # Lista para armazenar o MSE de cada Fold
r2s = [] # Lista para armazenar o R2 de cada Fold

print(f"Realizando o {n_splits}-fold cross-validation...")

# Definir o callback de Early Stopping
early_stopping_callback = EarlyStopping(
    monitor='loss',  # Monitorar a perda de validação
    patience=20,         # Número de épocas sem melhora após o qual o treinamento será interrompido
    restore_best_weights=True # Restaura os pesos do modelo da melhor época
)

# Loop iterando cada uma das pastas
for fold, (train_index, test_index) in enumerate(kf.split(data_X)): # Enumerando as pastas
    print(f"\nFold {fold+1}/{n_splits}")

    # Dividindo os dados de acordo com a pasta atual
    X_train_fold, X_test_fold = data_X[train_index], data_X[test_index]
    y_train_fold, y_test_fold = data_y[train_index], data_y[test_index]

    stdScaler = StandardScaler()
    minMax = MinMaxScaler()

    n_features = X_train_fold.shape[-1]     # 4

    X_train_reshaped = X_train_fold.reshape(-1, n_features) 
    X_test_reshaped = X_test_fold.reshape(-1, n_features) 
    
    X_train_scaled = stdScaler.fit_transform(X_train_reshaped)
    X_test_scaled = stdScaler.transform(X_test_reshaped)
    
    X_train_fold = X_train_scaled.reshape(X_train_fold.shape)
    X_test_fold = X_test_scaled.reshape(X_test_fold.shape)

    y_train_fold = minMax.fit_transform(y_train_fold.reshape(-1, 1)).flatten()
    y_test_fold = minMax.transform(y_test_fold.reshape(-1, 1)).flatten()

    # Inicializando o InceptionTimeRegressor com 100 épocas
    reg = InceptionTimeRegressor(n_epochs=100,
                                 batch_size=8,
                                 verbose = False,
                                 callbacks=[early_stopping_callback])
    # Treinando o Regressor, passando o callback e definindo uma parte para validação
    reg.fit(X_train_fold, y_train_fold)

    # Realizando as predições
    y_pred_fold = reg.predict(X_test_fold)

    # Calculando o MSE para a pasta atual
    mse_fold = mean_squared_error(y_test_fold, y_pred_fold)
    mses.append(mse_fold)

    # Calculando o R2 score para a pasta atual
    r2_fold = r2_score(y_test_fold, y_pred_fold)
    r2s.append(r2_fold)

    print(f"MSE para a pasta {fold+1}: {mse_fold:.4f}")
    print(f"R2 Score para a pasta {fold+1}: {r2_fold:.4f}")

# Calculando a média dos MSE e R2
average_mse = np.mean(mses)
average_r2 = np.mean(r2s)

print(f"\nAverage MSE across {n_splits} folds: {average_mse:.4f}")
print(f"Average R2 Score across {n_splits} folds: {average_r2:.4f}")


Realizando o 10-fold cross-validation...

Fold 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x728cd85869e0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 219m

# Salvar Resultado em Arquivo

In [5]:
import os

# Criando a pasta de resultados
os.makedirs('../results', exist_ok=True)

print(f"Pasta checada/criada com sucesso.")

Pasta checada/criada com sucesso.


In [6]:
import pandas as pd

# Criando a lista de fols + média
fold_labels = [f"Fold {i+1}" for i in range(len(mses))] + ["Average"]

# Criando um dicionário dos resultados
results_data = {
    'Fold': fold_labels,
    'MSE': mses + [average_mse],
    'R2 Score': r2s + [average_r2]
}

# Criando o DataFrame
results_df = pd.DataFrame(results_data)

# Salvando em CSV
results_df.to_csv("../results/inceptiontime_results.csv", index=False)

print("Resultados Salvos")

print("\nResumo dos Resultados:")
print(results_df)

Resultados Salvos

Resumo dos Resultados:
       Fold       MSE  R2 Score
0    Fold 1  0.042637 -0.101436
1    Fold 2  0.062524  0.418479
2    Fold 3  0.062118  0.084678
3    Fold 4  0.049975  0.099607
4    Fold 5  0.037675  0.301687
5    Fold 6  0.014044 -0.093635
6    Fold 7  0.053540 -0.310612
7    Fold 8  0.027664 -0.081193
8    Fold 9  0.024244  0.222236
9   Fold 10  0.043725  0.566519
10  Average  0.041815  0.110633
